In [1]:
using CSV
dfx = CSV.read("./Xvals.csv", delim=',', header=true)
dfy = CSV.read("./Yvals.csv", delim=',', header=true)
nb = size(dfx, 1);
A = convert(Matrix{Float64}, CSV.read("./affinitymatrix.csv", delim=',', header=true)[1:10, 2:11]);
X = convert(Matrix{Float64}, dfx)
Y = convert(Matrix{Float64}, dfy);

In [2]:
X = (X .- mean(X, 1)) ./ std(X, 1)
Y = (Y .- mean(Y, 1)) ./ std(Y, 1);

In [3]:
std(X, 1)

1×10 Array{Float64,2}:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [4]:
Φ = X * A * Y';

In [5]:
using Gurobi, JuMP

In [9]:
#primal
m = Model(solver=GurobiSolver())
@time @variable(m, π[1:nb, 1:nb] >= 0)

@time @constraint(m, distx[x=1:nb], sum(π[x, y] for y in 1:nb) <= 1/nb)
@time @constraint(m, disty[y=1:nb], sum(π[x, y] for x in 1:nb) <= 1/nb)

@time @objective(m, Max, sum(sum(π[x, y] * Φ[x, y] for x in 1:nb) for y in 1:nb));

  5.101258 seconds (9.55 M allocations: 356.765 MiB, 59.42% gc time)
  1.637998 seconds (2.88 M allocations: 137.622 MiB, 65.07% gc time)
  1.739765 seconds (2.88 M allocations: 137.637 MiB, 66.77% gc time)
  1.831803 seconds (4.19 M allocations: 118.359 MiB, 63.08% gc time)


In [10]:
@time solve(m)

Academic license - for non-commercial use only
Optimize a model with 2316 rows, 1340964 columns and 2681928 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-07, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e-04, 9e-04]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 0 rows and 671378 columns
Presolve time: 4.72s
Presolved: 2316 rows, 669586 columns, 1339172 nonzeros

Ordering time: 0.06s

Barrier statistics:
 AA' NZ     : 6.696e+05
 Factor NZ  : 1.428e+06 (roughly 300 MBytes of memory)
 Factor Ops : 1.073e+09 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   3.87021540e+04  5.82185209e-01  8.32e+01 5.58e+00  1.24e+00     7s
   1   8.71033545e+01  1.29966634e+01  2.09e-01 7.99e-15  3.51e-03     8s
   2   4.86601160e-01  1.28631877e+01  0.00e+00 7.88

:Optimal

In [55]:
#dual
m = Model(solver=GurobiSolver())
@variable(m, u[1:nb] >= 0)
@variable(m, v[1:nb] >= 0)

@constraint(m, blocking[x=1:nb, y=1:nb], u[x] + v[y] >= Φ[x, y])
@objective(m, Min, sum(u[x] for x in 1:nb) + sum(v[y] for y in 1:nb))
solve(m)

Academic license - for non-commercial use only
Optimize a model with 1340964 rows, 2316 columns and 2681928 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-07, 8e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 0 rows and 671378 columns (presolve time = 6s) ...
Presolve removed 0 rows and 671378 columns
Presolve time: 7.44s
Presolved: 2316 rows, 669586 columns, 1339172 nonzeros

Ordering time: 0.14s

Barrier statistics:
 AA' NZ     : 6.696e+05
 Factor NZ  : 1.428e+06 (roughly 300 MBytes of memory)
 Factor Ops : 1.073e+09 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   3.93757423e+04  6.74170472e+02  8.32e+01 5.58e+00  1.26e+00    10s
   1   6.81803951e+02  1.45621301e+04  0.00e+00 7.99e-15  

:Optimal